<a id='top'></a>
<img align="center" src="https://www.autotovari.com/wp-content/uploads/2017/08/Used-Cars-For-Sale.jpg" width="200" />
<p style="text-align: center;">
    <font size='5' type='bold'>
        <b>Определение стоимости автомобиля</b>
    </font>
</p>

# Содержание

<a href='#section_0'>Описание проекта</a>

<a href='#section_1'>1. Подготовка данных</a>

<a href='#section_2'>2. Обучение моделей</a>

* <a href='#section_2_1'>CatBoost</a>
* <a href='#section_2_2'>LightGBM</a>
* <a href='#section_2_3'>Linear Regression</a>

<a href='#section_3'>3. Анализ моделей</a>

<a href='#section_4'>Общий вывод</a>

<a href='#section_end'>Чек-лист готовности проекта</a>

<a id='section_0'></a>
# Описание проекта
Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

<div class="alert alert-info">
Привет! Меня зовут Никита Мишин и я буду твоим ревьюером по этому проекты.
Для простоты предлагаю общение на 'ты'. Буду предполагать, что ты не против:) 
Если более предпочтительно обращение на 'Вы', пиши, не стесняйся.
Также если будут возникать вопросы, аналогично, пиши:)

Предлагаю работать в известном тебе итеративном формате.
Итерация состоит в моей проверке твоего решения. 
После решения могут остаться какие-то недочеты, которые я попрошу тебя устранить, ты их исправляешь и я проверяю твои решения.
    Оставленные мною комментарии могут быть разного вида:
   
    - зеленый: элегантные решения, которые тебе стоит запомнит и в дальнейшем взять на вооружение:) 
    
    - желтый: сигнал о том, что есть некритичная вещь(не всегда ошибка), что нужно точно поправить в следующей работе, даже желательно в этой (полезно, в первую очередь, для тебя:) ).Также это рекомендации на будущее    

    - красный: недочет, который нужно исправить в этой работе, для того, чтобы она была принята
    
    - синий: полезная информация, доп ресурсы, "вопросы на подумать"

Также попрошу не удалять мои комментарии:) <a class="tocSkip">
</div>

<a id='section_1'></a>
## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer

<div class="alert alert-success">
Молодец, что группируешь все ипморты в одном месте, такая структуризации полезна и удобна
</div>

In [2]:
df = pd.read_csv('/datasets/autos.csv')
df.head(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


Набор данных представлен следующими признаками:
*	**DateCrawled** — дата скачивания анкеты из базы
*	**VehicleType** — тип автомобильного кузова
*	**RegistrationYear** — год регистрации автомобиля
*	**Gearbox** — тип коробки передач
*	**Power** — мощность (л. с.)
*	**Model** — модель автомобиля
*	**Kilometer** — пробег (км)
*	**RegistrationMonth** — месяц регистрации автомобиля
*	**FuelType** — тип топлива
*	**Brand** — марка автомобиля
*	**NotRepaired** — была машина в ремонте или нет
*	**DateCreated** — дата создания анкеты
*	**NumberOfPictures** — количество фотографий автомобиля
*	**PostalCode** — почтовый индекс владельца анкеты (пользователя)
*	**LastSeen** — дата последней активности пользователя

Целевой признак
*   **Price** — цена (евро)

Удалим признаки, не влияющие на предсказание цены: `PostalCode`, `DateCrawled`, `DateCreated`, `NumberOfPictures`, `LastSeen`.

<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
А что по поводу Postal code?
</div>

<div class="alert alert-info">
<h3>Комментарий студента <a class="tocSkip"></a></h3>
Done
</div>

<div class="alert alert-success">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
👍
</div>

In [4]:
df = df.drop(columns=['PostalCode', 'DateCrawled', 'LastSeen', 'DateCreated', 'NumberOfPictures'])

Видим присутствие пропусков в категориальных признаках. Заменим их на 'NaN'

In [5]:
df[['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']] = df[['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']].fillna('NaN')

In [6]:
# Избавимся от дубликатов

df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Да, молодец, что не забыл проверить на дубликаты
</div>

В признаках Power и RegistrationYear видим подозрительно высокие значения. Учитывая, что максимальной мощностью на данный момент в мире машин обладает Bugatti Chiron (1,479 л.с.), удалим все, что выше. Также поступим и с годами ранее 2021.

In [7]:
df = df[df['Power'] <= 1479]
df = df[df['RegistrationYear'] <= 2021]

<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
А что по поводу RegistrationYear? Тоже  какие-то аномальные значения
</div>

<div class="alert alert-info">
<h3>Комментарий студента <a class="tocSkip"></a></h3>
Done
</div>

<div class="alert alert-success">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
👍
</div>

In [8]:
# Посмотрим на общую информацию о данных

df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,326525.000000,326525.000000,326525.000000,326525.000000,326525.000000
mean,4404.907783,2002.903171,106.882863,128181.594059,5.697438
std,4518.527607,14.183144,67.029572,37890.960669,3.723373
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1000.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6350.000000,2008.000000,141.000000,150000.000000,9.000000
max,20000.000000,2019.000000,1436.000000,150000.000000,12.000000


In [9]:
# Приведем категориальные признаки в номинальные
# Применим OneHotEncoder для колонок VehicleType и FuelType

df = pd.get_dummies(df, columns=['VehicleType','FuelType'], drop_first=True)

# Применим LabelEncoder для колонок Brand, Model, Gearbox и NotRepaired

label_encode = LabelEncoder()
columns_for_convert = ['Brand', 'Model', 'Gearbox', 'NotRepaired']

for column in columns_for_convert:
    label_encode.fit(df[column].astype('str'))
    df[column] = label_encode.transform(df[column].astype('str'))


In [10]:
df.head()

,Price,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,Brand,NotRepaired,VehicleType_bus,...,VehicleType_small,VehicleType_suv,VehicleType_wagon,FuelType_cng,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol
0,480,1993,2,0,117,150000,0,38,0,0,...,0,0,0,0,0,0,0,0,0,1
1,18300,2011,2,190,26,125000,5,1,2,0,...,0,0,0,0,0,1,0,0,0,0
2,9800,2004,1,163,118,125000,8,14,0,0,...,0,1,0,0,0,1,0,0,0,0
3,1500,2001,2,75,117,150000,6,38,1,0,...,1,0,0,0,0,0,0,0,0,1
4,3600,2008,2,69,102,90000,7,31,1,0,...,1,0,0,0,0,1,0,0,0,0


<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
а ты уверен, что для всех столбцов подходит LE? Почему не используешь OHE?
</div>

<div class="alert alert-info">
<h3>Комментарий студента <a class="tocSkip"></a></h3>
Done
</div>

<div class="alert alert-success">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
👍
</div>

In [11]:
# Стандартизируем признаки с помощью StandardScaler

scaler = StandardScaler()
numeric = ['RegistrationYear','Power','Kilometer']
scaler.fit(df[numeric])
df[numeric] = scaler.transform(df[numeric])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326525 entries, 0 to 326825
Data columns (total 24 columns):
Price                      326525 non-null int64
RegistrationYear           326525 non-null float64
Gearbox                    326525 non-null int64
Power                      326525 non-null float64
Model                      326525 non-null int64
Kilometer                  326525 non-null float64
RegistrationMonth          326525 non-null int64
Brand                      326525 non-null int64
NotRepaired                326525 non-null int64
VehicleType_bus            326525 non-null uint8
VehicleType_convertible    326525 non-null uint8
VehicleType_coupe          326525 non-null uint8
VehicleType_other          326525 non-null uint8
VehicleType_sedan          326525 non-null uint8
VehicleType_small          326525 non-null uint8
VehicleType_suv            326525 non-null uint8
VehicleType_wagon          326525 non-null uint8
FuelType_cng               326525 non-null uint8
F

<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Да, верное решение о масштабировании признаков
</div>

Данные подготовлены для дальнейшей работы.

<a href='#top'>Back to top</a>

<a id='section_2'></a>
## Обучение моделей

In [13]:
# Выделим признаки для обучения и целевой признак
# для работы модели линейной регрессии

X = df.drop(columns=['Price'], axis=1)
y = df['Price']

In [14]:
# Разделим датасет на обучающую, тестовую и валидационную выборки
# в соотношении 60:20:20

rnd_state = 260686

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=rnd_state)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=rnd_state)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(195915, 23)
(65305, 23)
(65305, 23)
(195915,)
(65305,)
(65305,)


<div class="alert alert-danger">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Я все же попрошу тебя разделить на 3 части датасет. Обучающая, валидационную и тестовую
</div>

<div class="alert alert-info">
<h3>Комментарий студента <a class="tocSkip"></a></h3>
Done
</div>

<a id='section_2_1'></a>
### CatBoost

In [15]:
#вызовем CatBoost с нашими параметрами

cat_features = ['Gearbox', 'Model', 'Brand','NotRepaired']
model_cbr = CatBoostRegressor(cat_features=cat_features)

In [16]:
%%time
start = time.time()
model_cbr.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=200)
end = time.time()
catboost_time = end - start

0:	learn: 4428.7727478	test: 4450.6092439	best: 4450.6092439 (0)	total: 728ms	remaining: 12m 7s
200:	learn: 1976.5550446	test: 1992.0210162	best: 1992.0210162 (200)	total: 1m 55s	remaining: 7m 38s
400:	learn: 1873.0958589	test: 1896.2029823	best: 1896.2029823 (400)	total: 3m 54s	remaining: 5m 49s
600:	learn: 1826.0668939	test: 1858.0395409	best: 1858.0395409 (600)	total: 5m 56s	remaining: 3m 56s
800:	learn: 1798.6945586	test: 1838.3241800	best: 1838.3241800 (800)	total: 7m 51s	remaining: 1m 57s
999:	learn: 1777.7502130	test: 1823.1145502	best: 1823.1145502 (999)	total: 9m 45s	remaining: 0us

bestTest = 1823.11455
bestIteration = 999

CPU times: user 8min 47s, sys: 60 s, total: 9min 47s
Wall time: 9min 50s


In [17]:
start = time.time()
predictions = model_cbr.predict(X_valid)
end = time.time()
catboost_time_predicting = end - start
catboost_rmse = np.sqrt(((predictions - y_valid) ** 2).mean())
print('RMSE=', catboost_rmse)
print('Learning time=', catboost_time)
print('Predicting time=', catboost_time_predicting)

RMSE= 1823.1145501929789
Learning time= 590.2369091510773
Predicting time= 0.45092272758483887


<a href='#top'>Back to top</a>

<a id='section_2_2'></a>
### LightGBM

In [18]:
#вызовем LightGBM с нашими параметрами

model_lgb = lgb.LGBMRegressor()

In [19]:
%%time
start = time.time()
model_lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='rmse', categorical_feature=cat_features)
end = time.time()
lightgbm_time = end - start

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'Gearbox', 'Model', 'NotRepaired']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's rmse: 4219	valid_0's l2: 1.78e+07
[2]	valid_0's rmse: 3939.77	valid_0's l2: 1.55218e+07
[3]	valid_0's rmse: 3695.44	valid_0's l2: 1.36563e+07
[4]	valid_0's rmse: 3484.75	valid_0's l2: 1.21435e+07
[5]	valid_0's rmse: 3297.68	valid_0's l2: 1.08747e+07
[6]	valid_0's rmse: 3132.92	valid_0's l2: 9.81517e+06
[7]	valid_0's rmse: 2982.44	valid_0's l2: 8.89493e+06
[8]	valid_0's rmse: 2855.28	valid_0's l2: 8.15265e+06
[9]	valid_0's rmse: 2736.64	valid_0's l2: 7.4892e+06
[10]	valid_0's rmse: 2637.67	valid_0's l2: 6.95732e+06
[11]	valid_0's rmse: 2548.26	valid_0's l2: 6.49365e+06
[12]	valid_0's rmse: 2470.37	valid_0's l2: 6.10272e+06
[13]	valid_0's rmse: 2401.84	valid_0's l2: 5.76883e+06
[14]	valid_0's rmse: 2342.56	valid_0's l2: 5.48757e+06
[15]	valid_0's rmse: 2290.15	valid_0's l2: 5.2448e+06
[16]	valid_0's rmse: 2243.97	valid_0's l2: 5.0354e+06
[17]	valid_0's rmse: 2201.44	valid_0's l2: 4.84635e+06
[18]	valid_0's rmse: 2165.11	valid_0's l2: 4.68769e+06
[19]	valid_0's rmse: 2132.31

In [20]:
start = time.time()
predictions = model_lgb.predict(X_valid)
end = time.time()
lightgbm_time_predicting = end - start
lightgbm_rmse = np.sqrt(((predictions - y_valid) ** 2).mean())
print('RMSE=', lightgbm_rmse)
print('Learning time=', lightgbm_time)
print('Predicting time=', lightgbm_time_predicting)

RMSE= 1793.6851135623533
Learning time= 10.17509937286377
Predicting time= 0.8935346603393555


<a href='#top'>Back to top</a>

<a id='section_2_3'></a>
### Linear Regression

In [21]:
# Вызовем модель линейной регрессии

model_lr = LinearRegression()

In [22]:
%%time
start = time.time()
model_lr.fit(X_train,y_train)
end = time.time()
linear_time = end - start

CPU times: user 238 ms, sys: 96.2 ms, total: 334 ms
Wall time: 372 ms


In [23]:
start = time.time()
predictions = model_lr.predict(X_valid)
end = time.time()
linearregression_time_predicting = end - start
linearregression_rmse = np.sqrt(((predictions - y_valid) ** 2).mean())
print('RMSE=', linearregression_rmse)
print('Learning time=', linear_time)
print('Predicting time=', linearregression_time_predicting)

RMSE= 3312.8667577792935
Learning time= 0.37232422828674316
Predicting time= 0.006238698959350586


<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Да, отдельный лайк за то,что измеряешь время работы моделей!
</div>

<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Кстати, обучение моделей и предсказание: вот эту все логику можно оформить  в виде функции, для последующего переиспользования. Так код будет чище и в него будет легче вносить изменения. Плюс станет нагляднее_)
</div>

<div class="alert alert-danger">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Не хватает поиска оптимальных параметров модели через GridSearchCV. Несмотря на то,что ты побил бейзлайн, я тебя попрошу осуществить поиск гиперпараметров для закрепления теории
</div>

<div class="alert alert-info">
<h3>Комментарий студента <a class="tocSkip"></a></h3>
Done
</div>

<a href='#top'>Back to top</a>

<a id='section_3'></a>
## Анализ моделей

In [24]:
summary = pd.DataFrame(
    data={'CatBoost': [catboost_rmse, catboost_time, catboost_time_predicting],
          'LightGBM': [lightgbm_rmse, lightgbm_time, lightgbm_time_predicting],
         'LinearRegression': [linearregression_rmse, linear_time, linearregression_time_predicting]
         },
    index=['RMSE','LearningTime','PredictingTime']
)
summary

,CatBoost,LightGBM,LinearRegression
RMSE,1823.114550,1793.685114,3312.866758
LearningTime,590.236909,10.175099,0.372324
PredictingTime,0.450923,0.893535,0.006239


* Лучшее время обучения показала модель линейной регрессии - `0.33 сек`
* Лучшее время предсказания также показала модель линейной регрессии - `0.005 сек`, однако, и остальные модели укладываются по этому параметру в границах 1 сек
* Лучшую RMSE показала модель LightGBM - `1794`.

Попробуем улучшить модель LightGBM за счет вариации параметров. Проведем подбор параметров методом GridSearchCV.

In [25]:
# Проведем кросс-валидацию с помощью kFold с делением на 5 частей
cv = KFold(n_splits=5, shuffle=False)

# Выберем параметры для подбора оптимальных с GridSearchCV
param_grid = [
    {
        'n_estimators': [80,100],
        'num_leaves': [12,16,31],
        'subsample' : [0.75, 1.0],
    } 
]

grid = GridSearchCV(estimator=lgb.LGBMRegressor(), param_grid=param_grid, cv=cv)
grid.fit(X_train, y_train)
print(grid.best_estimator_)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=0.75, subsample_for_bin=200000, subsample_freq=0)


<div class="alert alert-success">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
👍
</div>

In [26]:
#вызовем LightGBM с нашими параметрами

model_lgb = lgb.LGBMRegressor(n_estimators=100, num_leaves=31, subsample=0.75)

In [27]:
%%time
start = time.time()
model_lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='rmse', categorical_feature=cat_features)
end = time.time()
lightgbm_time = end - start

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'Gearbox', 'Model', 'NotRepaired']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's rmse: 4219	valid_0's l2: 1.78e+07
[2]	valid_0's rmse: 3939.77	valid_0's l2: 1.55218e+07
[3]	valid_0's rmse: 3695.44	valid_0's l2: 1.36563e+07
[4]	valid_0's rmse: 3484.75	valid_0's l2: 1.21435e+07
[5]	valid_0's rmse: 3297.68	valid_0's l2: 1.08747e+07
[6]	valid_0's rmse: 3132.92	valid_0's l2: 9.81517e+06
[7]	valid_0's rmse: 2982.44	valid_0's l2: 8.89493e+06
[8]	valid_0's rmse: 2855.28	valid_0's l2: 8.15265e+06
[9]	valid_0's rmse: 2736.64	valid_0's l2: 7.4892e+06
[10]	valid_0's rmse: 2637.67	valid_0's l2: 6.95732e+06
[11]	valid_0's rmse: 2548.26	valid_0's l2: 6.49365e+06
[12]	valid_0's rmse: 2470.37	valid_0's l2: 6.10272e+06
[13]	valid_0's rmse: 2401.84	valid_0's l2: 5.76883e+06
[14]	valid_0's rmse: 2342.56	valid_0's l2: 5.48757e+06
[15]	valid_0's rmse: 2290.15	valid_0's l2: 5.2448e+06
[16]	valid_0's rmse: 2243.97	valid_0's l2: 5.0354e+06
[17]	valid_0's rmse: 2201.44	valid_0's l2: 4.84635e+06
[18]	valid_0's rmse: 2165.11	valid_0's l2: 4.68769e+06
[19]	valid_0's rmse: 2132.31

In [28]:
start = time.time()
predictions = model_lgb.predict(X_valid)
end = time.time()
lightgbm_time_predicting = end - start
lightgbm_rmse = np.sqrt(((predictions - y_valid) ** 2).mean())
print('RMSE=', lightgbm_rmse)
print('Learning time=', lightgbm_time)
print('Predicting time=', lightgbm_time_predicting)

RMSE= 1793.6851135623533
Learning time= 10.202895164489746
Predicting time= 0.898681640625


In [29]:
# Посмотрим на качество модели LightGBM на тестовой выборке

model_lgb_test = lgb.LGBMRegressor(n_estimators=100, num_leaves=31, subsample=0.75)

In [30]:
%%time
start = time.time()
model_lgb_test.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='rmse', categorical_feature=cat_features)
end = time.time()
lightgbm_test_time = end - start

[1]	valid_0's rmse: 4203.7	valid_0's l2: 1.76711e+07
[2]	valid_0's rmse: 3928.67	valid_0's l2: 1.54344e+07
[3]	valid_0's rmse: 3687.09	valid_0's l2: 1.35946e+07
[4]	valid_0's rmse: 3477.07	valid_0's l2: 1.209e+07
[5]	valid_0's rmse: 3291.53	valid_0's l2: 1.08341e+07
[6]	valid_0's rmse: 3127.23	valid_0's l2: 9.77954e+06
[7]	valid_0's rmse: 2978.27	valid_0's l2: 8.87008e+06
[8]	valid_0's rmse: 2852.51	valid_0's l2: 8.13683e+06
[9]	valid_0's rmse: 2735.12	valid_0's l2: 7.4809e+06
[10]	valid_0's rmse: 2636.75	valid_0's l2: 6.95245e+06
[11]	valid_0's rmse: 2548.45	valid_0's l2: 6.4946e+06
[12]	valid_0's rmse: 2470.94	valid_0's l2: 6.10552e+06
[13]	valid_0's rmse: 2402.49	valid_0's l2: 5.77195e+06
[14]	valid_0's rmse: 2342.92	valid_0's l2: 5.48926e+06
[15]	valid_0's rmse: 2291.05	valid_0's l2: 5.24892e+06
[16]	valid_0's rmse: 2244.82	valid_0's l2: 5.03923e+06
[17]	valid_0's rmse: 2203.21	valid_0's l2: 4.85411e+06
[18]	valid_0's rmse: 2166.83	valid_0's l2: 4.69514e+06
[19]	valid_0's rmse: 213

In [31]:
start = time.time()
predictions = model_lgb_test.predict(X_test)
end = time.time()
lightgbm_test_time_predicting = end - start
lightgbm_test_rmse = np.sqrt(((predictions - y_test) ** 2).mean())
print('RMSE (test)=', lightgbm_test_rmse)
print('Learning time (test)=', lightgbm_test_time)
print('Predicting time (test)=', lightgbm_test_time_predicting)

RMSE (test)= 1792.5817640057433
Learning time (test)= 10.771301984786987
Predicting time (test)= 0.8947010040283203


Параметры сопоставимы. Модель хорошо показала себя и на тестовой выборке,

<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Бэйзлайн по качеству модели смог пройти, молодец
</div>

<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Отдельный плюс в карму за вывод и табличку с результатами:) Очень наглядно
</div>

<a id='section_4'></a>
# Общий вывод
В результате анализа модели показали различные значения по времени работы и среднеквадратичной ошибке. Наиболее точными оказались модели, использующие градиентный бустинг. Значение ошибки у этих моделей практически в два раза меньше,чем у модели линейной регрессии. При этом модель LightGBM значительно превзошла модель CatBoost по скорости обучения.

Модель LightGBM является оптимальной для применения на данном наборе данных.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

-Спасибо за качественно сделанный проект, было приятно проверять. 

- Видно, что приложено много усилий

- Код написан хорошо 

- Соблюдена структура проекта 
    
- Твои модели смогли достичь хорошего качества, поздравляю!
    
- К сожалению есть пара моментов, которые необходимо поправить, чтобы проект был принят:
    
    - нет поиска гиперпараметров
    
    - отсутствует разбиение на тестовую. обучающую и валидационную выборки

- Также оставил пару замечаний и советов


Жду твоих исправлений!

</div>

<a href='#top'>Back to top</a>

<a id='section_end'></a>
## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей

<a href='#top'>Back to top</a>